# Ayudantía 11: Serialización y Networking 2 :inbox_tray:

## Ayudantes 👾

Y sus recomendaciones semanales 🎵

- S1: Enzo Acosta
  - [Bright Lies - Giant Rooks](https://www.youtube.com/watch?v=Cm2RgfwFZDg)
- S2: Bastián Pérez
  - [American Pie - Don McLean](https://www.youtube.com/watch?v=PRpiBpDy7MQ)
- S3: Clemente Campos
  - [Junk - Paul McCartney](https://www.youtube.com/watch?v=3svS8Nk6-Jk)
- S4: Carlos Olguín
  - [Skeletons - Travis Scott](https://www.youtube.com/watch?v=tAyYYKcySXA)
- S5: Carlos Martel
  - [First of the Year - Skrillex](https://www.youtube.com/watch?v=2cXDgFwE13g)

Con aparición especial de:

- Daniela Concha
  - [Mercury - Sufjan Stevens, Bryce Dessner, Nico Muhly, James McAlister](https://www.youtube.com/watch?v=rmZKVOaTc68)

## Contenidos 📖

- Manejo de bytes
- Networking mediante sockets
- Cliente/Servidor TCP

## Introducción

En esta ayudantía continuaremos explorando en los conceptos de serialización y networking, pero esta vez en mayor profundidad. Haremos uso de sockets para la comunicación entre cliente y servidor mediante el protocolo TCP, y realizaremos la gestión de los bytes que serán comunicados en la conexión.

## DCClassroomServer

Luego de desarrollar DCCPuchamon te has dado cuenta que la comunicación entre cliente y servidor no es tan simple como parece. Por lo que decides crear una nueva aplicación llamada DCClassroomServer, la cual permitirá a los estudiantes conectarse a un servidor centralizado en el que podrán subir sus tareas, descargar material de estudio, ver a la lista de estudiantes del curso y comunicarse con ellos.

Para lograrlo, debes tener en cuenta el protocolo de aplicación que usarás entre el cliente y el servidor.

El cliente debe enviar **solicitudes** al servidor en formato JSON, y con los siguientes atributos:

  - `username`: Nombre del usuario del estudiante que realiza la solicitud.
  
  - `comando`: Indica la operación que se desea realizar. Puede ser ``'carga'`` (subir un archivo), ``'descarga'`` (descargar un archivo), ``'listar_usuarios'`` (obtener la lista de estudiantes conectados) o ``'mensaje_directo'`` (enviar un mensaje a otro estudiante).

  - `file_name`: Nombre del archivo a subir o descargar. Incluir en el caso de las operaciones ``'listar_usuarios'`` y ``'mensaje_directo'``.

  - `file_content_b64`: Contenido del archivo codificado en base64 _(*)_. Solo aplicable para la operación ``'carga'``.
  
  - `destinatario`: Nombre del usuario al que se desea enviar un mensaje. Solo aplicable para la operación ``'mensaje_directo'``.
  
  - `mensaje`: Mensaje a enviar al estudiante. Solo incluir para la operación ``'mensaje_directo'``.

El servidor debe **responder** a las solicitudes del cliente de la siguiente forma:

- Si se lanza un error, debe un JSON con los siguientes atributos:
  - `tipo`: Indica el tipo de respuesta. En este caso, siempre será ``'error'``.
  - `descripcion`: Mensaje descriptivo del error ocurrido.

- Si todo sale bien, el servidor debe responder con un JSON con los siguientes atributos:
  
  - Si el comando fue ``'carga'``, el servidor debe responder con:
    - `tipo`: ``'confirmacion'``.
    - `descripcion`: Mensaje confirmando que el archivo fue recibido correctamente.
  
  - Si el comando fue ``'descarga'``, el servidor debe responder con:
    - `username`: Corresponde a ``'Servidor'``.
    - `tipo`: ``'archivo_descargado'``.
    - `file_name`: Nombre del archivo descargado.
    - `file_content_b64`: Contenido del archivo descargado codificado en base64 _(*)_.
  
  - Si el comando fue ``'listar_usuarios'``, el servidor debe responder con:
    - `username`: Corresponde a ``'Servidor'``.
    - `tipo`: ``'lista_usuarios'``.
    - `usuarios`: Lista de nombres de usuarios conectados.
  
  - Si el comando fue ``'mensaje_directo'``, el servidor debe enviar al destinatario un JSON con:
    - `username`: Corresponde a ``'Servidor'``.
    - `tipo`: ``'mensaje_entrante'``.
    - `remitente`: Nombre del usuario que envió el mensaje.
    - `mensaje`: Mensaje recibido.
    Y responder al remitente con:
    - `tipo`: ``'confirmacion'``.
    - `descripcion`: Mensaje confirmando que el mensaje fue enviado correctamente.

_(*) Se utiliza base64 para codificar los datos binarios a una representación de texto que pueda ser incluída en el JSON._

## Ejercicio 1: Servidor

Deberás completar la clase `Server` (en el archivo `server.py`) para manejar la conexión con cada estudiante. Asegúrate de manejar correctamente los errores y cerrar las conexiones una vez que la operación haya finalizado. Especificamente deberas completar:
- **`client_connection(self, client_socket: socket.socket, addr: tuple)`**:
  
  - Este método debe escuchar la conexión del cliente, recibir el tamaño del mensaje, los bytes de la request, deserializar el JSON y manejar los diferentes comandos (`carga`, `descarga`, `listar_usuarios`, `mensaje_directo`) según corresponda. Finalmente, debe cerrar el socket del cliente y eliminar su información de la lista de clientes conectados. En cada comando se debe realizar lo siguiente:
  
    - En el caso de `carga`, se debe verificar que se hayan incluído los atributos `file_name` y `file_content_b64`. Si alguno falta, se debe enviar un mensaje de error al cliente. Si ambos están presentes, se debe decodificar el contenido del archivo desde base64 y guardarlo como archivo en el servidor.
  
    - En el caso de `descarga`, se debe verificar que el archivo solicitado exista en el servidor. Si no existe, se debe enviar un mensaje de error al cliente. Si existe, se debe leer el archivo, codificar su contenido en base64 y enviarlo al cliente.
  
    - Si el comando es `listar_usuarios`, se debe enviar al cliente la lista de usuarios conectados.
  
    - Si el comando es `mensaje_directo`, se hayan incluído los atributos `destinatario` y `mensaje`. Si alguno falta, se debe enviar un mensaje de error al cliente. Si ambos están presentes, se debe obtener el socket del destinatario y enviarle el mensaje directo. Luego, se debe enviar una confirmación al remitente.
  
- **`send_data(self, data: str, client_socket: socket.socket)`**:

  - Este método debe pasar la data a enviar a bytes, obtener su tamaño y enviar primero el tamaño (4 bytes) y luego la data como tal.

## Ejercicio 2: Cliente

Ademas deberas completar por la parte del `Cliente` el cual buscara conectarse al servidor y enviar diversos tipos de mensajes especificos mediante la consola, especificamente:
- Enviar y recibir archivos (`subir` / `descargar`).
- Solicitar y ver la lista de todos los alumnos conectados (`lista`).
- Enviar mensajes directos a un alumno específico (`msg`).
- Identificarse ante el servidor al conectarse (esto lo hace `connect_to_server` al enviar el `"saludo"`, que a su vez usa `send_json`).

A continuación, se explica la lógica de las funciones que implementan esto:

- **`send_json(self, msg: dict)`**
    - Esta es la función central de envío. Se encarga de convertir el diccionario de Python (`msg`) en un string JSON, codificarlo a bytes (`UTF-8`), calcular su longitud, y enviar primero la longitud (4 bytes) y luego los datos (el JSON) al servidor. Siempre se asegura de que el `username` del cliente (`self.nombre_cliente`) esté en el mensaje antes de enviarlo.

- **`mandar_solicitud_archivo(self, tipo, nombre_archivo)`**
    - **Si `tipo == "carga"`:** Esta parte lee un archivo local (`nombre_archivo`) desde el disco en modo binario (`'rb'`). Luego, codifica esos bytes en Base64 (un formato de texto seguro para JSON) y lo envía al servidor dentro de un JSON con la operación `"carga"`, el nombre del archivo (`file_name`) y el contenido ya codificado en `"file_content_b64"`.
    - **Si `tipo == "descarga"`:** Esta parte es más simple. Solo envía un JSON al servidor con la operación `"descarga"` y el `file_name` que se quiere obtener. El servidor buscará el archivo y, si lo encuentra, responderá con un mensaje `archivo_descargado`, que será procesado por `decode_msg_from_server`.

- **`mandar_solicitud_alumnos(self)`**
    - Esta función construye y envía un JSON muy simple al servidor. El JSON solo contiene la operación `"listar_usuarios"`. El servidor, al recibir esto, responderá con un mensaje tipo `"lista_usuarios"` que incluye a todos los clientes conectados, y `decode_msg_from_server` lo imprimirá en la consola.

- **`mandar_mensaje_alumno(self, destinado, msg)`**
    - Esta función empaqueta un JSON para el servidor con la operación `"mensaje_directo"`, el nombre del `"destinatario"` (a quién va el mensaje) y el `"mensaje"` en sí. El servidor se encarga de buscar el socket de ese destinatario y reenviarle el mensaje.

_Nota: Los mensajes deben enviarse con endianess 'big'._